<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>
<h1> Question  #1: </h1>

<b>Obtaining DataFrame from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M .</b>
</div>

In [194]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [195]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [196]:
MyTable =soup.find_all('table')                     #reads all tables in the web site
MyTable = MyTable[0]                                #our target table is the first table in the web site accoring to inspection of the site
mylist = list()
with open ('toronto.txt', 'w') as r:                # to write the result data set for future use
    for row in stat_table.find_all('tr'):           #reads each row
        for cell in row.find_all('td'):             #finds each cell in the specitied row
            cell.text.strip()                       # to get rid of leading and tailing spaces   
            #print(cell.text)
            if cell.text[4:16] == "Not assigned":   # to skip  "Not assigned" cells
                continue
            a1 = cell.text[1:4]
            neighborhood = cell.text.replace('/',',')  # It's seen that neighborhood list is defined between parentheses and separated by "/"
            neighborhoodlist = neighborhood.split('(') # It's seen that neighborhood list is defined between parentheses and separated by "/"
            a2 = neighborhoodlist[0][4:]
            a3 = neighborhoodlist[1][:-3]
            a3 = a3.replace(' ,',',')
            mylist.append([a1,a2, a3])
            newrecord=a1+", "+a2+", "+a3+"\n"
            r.write(newrecord)
headers = ["PostalCode", "Borough", "Neighborhood"]
df=pd.DataFrame(mylist)
df[1][100]="East Toronto"           #Secon cell of the 100.row is an exception for the rule which is explained above. So that It's manually corrected 
df.columns=headers
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [197]:
df.shape

(103, 3)